In [71]:
import os
import re
import pandas as pd
import json
import xml.etree.ElementTree as ET
from collections import Counter
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
from random import randrange, sample, seed
seed(1)
import numpy as np

In [4]:
def isDelays(sentence):
    if len(sentence) < 100 and ('?' in sentence or ' if ' in sentence):
        return False
    if ' wait ' in sentence \
    or ' delay ' in sentence \
    or ' slow ' in sentence \
    or ' for so long ' in sentence \
    or ' be so long ' in sentence \
    or ' take so long ' in sentence \
    or ' long time ' in sentence \
    or ' longer ' in sentence \
    or ' postpone ' in sentence \
    or ' reschedule ' in sentence \
    or ' numerous time' in sentence \
    or ' quicker ' in sentence \
    or '  take forever' in sentence \
    or ' prolong ' in sentence:
        return True
    if (' ridiculous ' in sentence 
        or ' unacceptable 'in sentence) and \
    ('months' in sentence \
     or 'days' in sentence \
     or 'weeks' in sentence \
     or 'hours' in sentence \
     or 'years' in sentence \
     or 'month' in sentence \
     or 'day' in sentence \
     or 'week' in sentence \
     or 'hour' in sentence \
     or 'year' in sentence):
        return True
    return False

def isCosts(sentence):
    if len(sentence) < 100 and ('?' in sentence or ' if ' in sentence):
        return False
    if ' cost ' in sentence \
    or ' costly ' in sentence \
    or ' pay for ' in sentence \
    or 'payment' in sentence \
    or 'afford' in sentence \
    or 'expensive' in sentence \
    or 'cheap' in sentence \
    or ' money ' in sentence \
    or 'limit' in sentence \
    or ' overprice ' in sentence \
    or ' price' in sentence \
    or ' copay' in sentence \
    or ' bill ' in sentence \
    or ' co-pay ' in sentence \
    or ' fee ' in sentence \
    or ' charge ' in sentence \
    or ' deductible ' in sentence \
    or ' premium ' in sentence \
    or 'out-of-network' in sentence \
    or 'out of network' in sentence \
    or 'in-network' in sentence \
    or 'in network' in sentence:
        return True
    return False

def isAccess(sentence):
    if len(sentence) < 100 and ('?' in sentence or ' if ' in sentence):
        return False
    if ' access to ' in sentence \
    or ' be refuse ' in sentence \
    or ' accept new patient ' in sentence \
    or ' be cancel ' in sentence \
    or 'consultation' in sentence \
    or 'registration' in sentence \
    or 'shortage' in sentence \
    or ' crowd ' in sentence \
    or ' reschedule ' in sentence \
    or 'cancellation' in sentence \
    or ' ward ' in sentence \
    or 'available' in sentence \
    or 'reject' in sentence \
    or 'to service' in sentence \
    or 'admission' in sentence \
    or 'out-of-network' in sentence \
    or 'out of network' in sentence \
    or 'in-network' in sentence \
    or 'in network' in sentence \
    or 'limitation' in sentence \
    or ' nearby hospital' in sentence \
    or ' nearby clinic ' in sentence:
        return True
    if ' limit ' in sentence \
    and 'access' in sentence:
        return True
    if ' treat' in sentence \
    and ' refus' in sentence:
        return True
    return False

     
def isErrors(sentence):
    if len(sentence) < 100 and ('?' in sentence or ' if ' in sentence):
        return False
    if ' inept ' in sentence and 'doctor' in sentence:
        return True
    if 'miss diagnosis' in sentence \
    or 'mis-diagnosis' in sentence \
    or 'misdiagnose' in sentence \
    or 'misdiagnosis' in sentence \
    or 'clerical' in sentence \
    or 'erroneous' in sentence \
    or 'wrong disease' in sentence \
    or 'typo' in sentence \
    or 'wrong medi' in sentence \
    or 'medical accident' in sentence \
    or 'incorrect' in sentence \
    or 'inconclusive' in sentence \
    or 'fault' in sentence \
    or 'mislead' in sentence \
    or 'examination' in sentence \
    or 'incompetent' in sentence \
    or 'inept' in sentence \
    or 'judgement' in sentence \
    or 'judgment' in sentence \
    or 'miss' in sentence:
        return True
    if ('make' and 'mistake' in sentence) \
    or ' be a mistake' in sentence:
        return True
    return False

def isTreatments(sentence):
    if len(sentence) < 100 and ('?' in sentence or ' if ' in sentence):
        return False
    if ' get better ' in sentence \
    and ' you ' not in sentence \
    and ' hope' not in sentence \
    and ' life ' not in sentence \
    and ' will ' not in sentence \
    and '?' not in sentence \
    and ' if ' not in sentence \
    and ' to get better ' not in sentence \
    and ' can get better ' not in sentence \
    and ' it ll ' not in sentence \
    and ' i ' in sentence:
        return True
    if ' feel better ' in sentence \
    and ' you ' not in sentence \
    and ' hope' not in sentence \
    and ' will ' not in sentence \
    and '?' not in sentence \
    and ' if ' not in sentence \
    and ' can feel better ' not in sentence \
    and ' would ' not in sentence \
    and ' should feel better' not in sentence \
    and ' i ' in sentence:
        return True
    if ' get worse ' in sentence \
    and ' you ' not in sentence \
    and ' hope' not in sentence \
    and ' life ' not in sentence \
    and ' will ' not in sentence \
    and '?' not in sentence \
    and ' if ' not in sentence \
    and ' to get worse ' not in sentence \
    and ' can get worse ' not in sentence \
    and ' it ll ' not in sentence \
    and ' i ' in sentence:
        return True
    if ' feel worse ' in sentence \
    and ' you ' not in sentence \
    and ' hope' not in sentence \
    and ' will ' not in sentence \
    and '?' not in sentence \
    and ' if ' not in sentence \
    and ' can feel worse ' not in sentence \
    and ' would ' not in sentence \
    and ' should feel worse ' not in sentence \
    and ' i ' in sentence:
        return True

    if ' treatment ' in sentence \
    and ' hope' not in sentence:
        return True

    if ' be never end ' in sentence \
    or ' relapse ' in sentence \
    or ' still sick' in sentence \
    or ' still ill' in sentence \
    or ' recurrent ' in sentence \
    or ' constant pain ' in sentence \
    or ' be cure ' in sentence \
    or ' assessment ' in sentence \
    or ' rehabilitation ' in sentence \
    or ' good therapy ' in sentence \
    or ' great therapy ' in sentence \
    or ' excellent therapy ' in sentence \
    or ' prognosis ' in sentence \
    or (' therapy '  in sentence and ' not work '  in sentence):
        return True
    return False



def isStaffAndTrust(sentence):
    if len(sentence) < 100 and ('?' in sentence or ' if ' in sentence):
        return False
    # staff attitude and patient trust
    if 'manager' in sentence \
    or 'staff' in sentence \
    or ' nurse' in sentence \
    or 'therapist' in sentence \
    or 'attitude' in sentence \
    or 'unethical' in sentence \
    or 'ethical' in sentence \
    or ' be care' in sentence \
    or 'uncaring' in sentence \
    or 'obnoxious' in sentence \
    or 'lovely' in sentence \
    or 'dismissive' in sentence \
    or ' so kind' in sentence \
    or 'cruel' in sentence \
    or 'disrespectful' in sentence \
    or ' respectful ' in sentence \
    or ' empathetic ' in sentence \
    or ' empathy ' in sentence \
    or ' sympathy ' in sentence \
    or 'sympathetic' in sentence \
    or ' trust' in sentence \
    or 'lack of concern' in sentence \
    or ' rude' in sentence \
    or ' dismiss me' in sentence \
    or 'lack of respect' in sentence \
    or 'polite' in sentence \
    or ' uncaring ' in sentence \
    or ' manner' in sentence \
    or 'so mean' in sentence \
    or 'aggressive' in sentence \
    or 'insensitive' in sentence \
    or 'neglect' in sentence \
    or 'lack of understand' in sentence \
    or 'confidence' in sentence \
    or 'privacy' in sentence:
        return True
    if ' be understand ' in sentence \
    and 'husband' not in sentence \
    and 'wife' not in sentence \
    and 'employer' not in sentence \
    and 'boyfriend' not in sentence \
    and 'girlfriend' not in sentence \
    and 'i be understand' not in sentence \
    and 'family' not in sentence:
        return True

    # staff’s professional skills and conduct
    if ' skill' in sentence \
    or 'treat well' in sentence \
    or 'treat bad' in sentence \
    or 'unclear' in sentence \
    or ' good treat ' in sentence \
    or ' great treat ' in sentence \
    or ' poor treat ' in sentence \
    or ' bad treat ' in sentence \
    or ' worst treat ' in sentence \
    or ' good care' in sentence \
    or ' great care' in sentence \
     or ' excellent care' in sentence \
    or ' poor care' in sentence \
    or ' bad care' in sentence \
    or ' inept ' in sentence \
    or 'poor communication' in sentence \
    or ' expertise' in sentence \
    or 'fob off' in sentence \
    or 'helpful' in sentence \
    or 'unprofessional' in sentence \
    or 'shout at' in sentence \
    or 'irresponsible' in sentence \
    or 'unqualified' in sentence \
    or 'qualify' in sentence \
    or 'lack of train' in sentence \
    or 'no train' in sentence \
    or 'incapable' in sentence \
    or 'unwilling' in sentence \
    or 'corrupt' in sentence \
    or 'ignore' in sentence \
    or 'mislead' in sentence \
    or 'lack of knowledge' in sentence \
    or ' bad behavi' in sentence \
    or 'not listen' in sentence \
    or 'inappropriate' in sentence \
    or 'dedicate' in sentence \
    or 'very professional' in sentence \
    or 'so professional' in sentence:
        return True
    if ' look after me' in sentence \
    and 'friend' not in sentence \
    and 'husband' not in sentence \
    and 'wife' not in sentence:
        return True
    return False

def hasCourse(sentence, cou_realtions):
    if len(cou_realtions) > 0 \
    and 'hope' not in sentence \
    and '?' not in sentence \
    and ' will ' not in sentence \
    and ' if ' not in sentence \
    and ' you ' not in sentence:
        return True
    return False

def processForRules(sen):
    lemmatizer = WordNetLemmatizer()
    new_sen = ""
    for word in sen.lower().split():
        new_word = lemmatizer.lemmatize(word, pos='v')
        new_word = lemmatizer.lemmatize(new_word, pos='n')
        new_sen += ' ' + new_word
    return new_sen[1:]

In [5]:
def parseSentense(lines, root, text):
    sen_index = []
    treatment_index = []
    test_index = []
    cou_of_index = []
    sentence_split = []
    treatments = []
    tests = []
    cou_of = []
    for line in lines:
        if line[:11] == 'NamedEntity' and line[31:40] == 'treatment':
            treatment_index.append([int(line.split("\t")[1]), int(line.split("\t")[2]), line.split("\t")[7][3:].strip()])
        if line[:11] == 'NamedEntity' and line[31:35] == 'test':
            test_index.append([int(line.split("\t")[1]), int(line.split("\t")[2]), line.strip().split("\t")[7][3:].strip()])
        if line[:8] == 'Sentence':
            sen_index.append([int(line.split("\t")[1]), int(line.split("\t")[2])])
    
    entity_dict = {}
    for child in root:
        if 'ClampNameEntityUIMA' in child.tag and child.attrib['semanticTag'] in set(["COU", "problem"]):
            entity_dict[child.attrib['{http://www.omg.org/XMI}id']] = {"begin-end": (int(child.attrib['begin']), int(child.attrib['end'])), "Semantic": child.attrib['semanticTag']}
            
        if 'ClampRelationUIMA' in child.tag and child.attrib['semanticTag'] == "COU_Of":
            cou_of_index.append((entity_dict[child.attrib['entTo']]["begin-end"][0],
                                entity_dict[child.attrib['entTo']]["begin-end"][1], 
                                entity_dict[child.attrib['entFrom']]["begin-end"][0],
                                entity_dict[child.attrib['entFrom']]["begin-end"][1]))
    for index in sen_index:
        sentence_split.append(text[index[0]:index[1]])
        treatment = []
        for t_index in treatment_index:
            if t_index[0] >= index[0] and t_index[1] <= index[1]:
                treatment.append(t_index[2])
        treatments.append(treatment)
                                
        test = []
        for te_index in test_index:
            if te_index[0] >= index[0] and te_index[1] <= index[1]:
                test.append(te_index[2])
        tests.append(test)
        
        cou = []
        for cou_index in cou_of_index:
            if cou_index[0] >= index[0] \
            and cou_index[1] <= index[1] \
            and cou_index[2] >= index[0] \
            and cou_index[3] <= index[1]:
                cou.append(text[cou_index[0]:cou_index[1]] + '->' + text[cou_index[2]:cou_index[3]])
        cou_of.append(cou)
    return (sentence_split, treatments, tests, cou_of)

In [6]:
raw_text_path = "/Users/jaden/ClampCmd_1.6.0/input"
entity_annotation_path = "/Users/jaden/ClampCmd_1.6.0/output"
sentense_dic  = []
countforannotation = {'isDelays': 0, 'isCosts': 0, 'isAccess': 0, 'isErrors': 0, 'isTreatments': 0, 'isStaffAndTrust': 0}

for files in sorted(os.listdir(raw_text_path)):
    if '.' not in files:
        print(files)
        for filename in os.listdir(raw_text_path + '/' + files):
            if filename[-4:] == ".txt":
                text_f = open(raw_text_path + '/' + files + '/' + filename, 'r')
                endIndex = re.search('-',filename).span()[0]
                group = filename[:endIndex]
                text = text_f.readline()
                text_f.close()
            
                anno_f = open(entity_annotation_path + '/' + filename[0] + "/" + filename, 'r')
                tree = ET.parse(entity_annotation_path + '/' + filename[0] + "/" + filename[:-4] + '.xmi')
                root = tree.getroot()
                sentences, treatments, test, cou_realtions = parseSentense(anno_f.readlines(), root, text)
                anno_f.close()
                count = 0
                for sen, treat, te, cou_realtion in zip(sentences, treatments, test, cou_realtions):
                    count += 1
                    trainOrtest = 'test'
                    train_class = []
                    processed_sen = processForRules(sen)
                    if isDelays(processed_sen):
                        countforannotation['isDelays'] += 1
                        train_class.append(1)
                        trainOrtest = 'train'
                    if isCosts(processed_sen):
                        countforannotation['isCosts'] += 1
                        train_class.append(2)
                        trainOrtest = 'train'
                    if isAccess(processed_sen):
                        countforannotation['isAccess'] += 1
                        train_class.append(3)
                        trainOrtest = 'train'
                    if isErrors(processed_sen):
                        countforannotation['isErrors'] += 1
                        train_class.append(4)
                        trainOrtest = 'train'
                    if isTreatments(processed_sen) or hasCourse(processed_sen, cou_realtion):
                        countforannotation['isTreatments'] += 1
                        train_class.append(5)
                        trainOrtest = 'train'
                    if isStaffAndTrust(processed_sen):
                        countforannotation['isStaffAndTrust'] += 1
                        train_class.append(6)
                        trainOrtest = 'train'
                    sentense_dic.append({'group': group, 'id': filename[:-4] + '-' +str(count), 'text': sen, 'treatment': treat, 'course_of_problem': cou_realtion, 'test': te, 'trainOrtest': trainOrtest, 'aspect': train_class})


A
B
C
D
E
F
G
H
I
K
L
M
N
O
P
Q
R
S
T
U
V
W


In [7]:
whole_df = pd.DataFrame(sentense_dic)

In [8]:
whole_df.head()

,aspect,course_of_problem,group,id,test,text,trainOrtest,treatment
0,[],[],Ankle_Problems,Ankle_Problems-656172-3-1,[],"What I find weird is , even with no pain I am ...",test,[]
1,[],[],Ankle_Problems,Ankle_Problems-656172-3-2,[],I guess it will take awhile and more practice ...,test,[]
2,[5],[reduce->swelling],Ankle_Problems,Ankle_Problems-656172-3-3,[],Yeah I will continue with therapy and also usi...,train,[]
3,[],[],Anxiety_Disorders,Anxiety_Disorders-591664-1-1,[],I am also on it and at similar levels .,test,[]
4,[],[],Anxiety_Disorders,Anxiety_Disorders-591664-1-2,[],My husband thinks this is what is making the d...,test,[]


In [9]:
whole_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7651255 entries, 0 to 7651254
Data columns (total 8 columns):
aspect               object
course_of_problem    object
group                object
id                   object
test                 object
text                 object
trainOrtest          object
treatment            object
dtypes: object(8)
memory usage: 467.0+ MB


In [12]:
whole_df.to_csv('whole_data_sentence.csv')

In [10]:
print('Total sentences to be annotaed:', sum(countforannotation.values()))
countforannotation

Total sentences to be annotaed: 601424


{'isDelays': 153801,
 'isCosts': 64082,
 'isAccess': 29356,
 'isErrors': 52341,
 'isTreatments': 154615,
 'isStaffAndTrust': 147229}

In [84]:
isDelays_ = []
isCosts_ = []
isAccess_ = []
isErrors_ = []
isTreatments_ = []
isStaffAndTrust_ = []
for each in sentense_dic:
    if 1 in each['aspect']:
        isDelays_.append(each)
    if 2 in each['aspect']:
        isCosts_.append(each)
    if 3 in each['aspect']:
        isAccess_.append(each)
    if 4 in each['aspect']:
        isErrors_.append(each)
    if 5 in each['aspect']:
        isTreatments_.append(each)
    if 6 in each['aspect']:
        isStaffAndTrust_.append(each)
        
isDelays_df = pd.DataFrame(isDelays_)
isCosts_df = pd.DataFrame(isCosts_)
isAccess_df = pd.DataFrame(isAccess_)  
isErrors_df = pd.DataFrame(isErrors_)  
isTreatments_df = pd.DataFrame(isTreatments_)  
isStaffAndTrust_df = pd.DataFrame(isStaffAndTrust_) 

# isDelays_df['sub_aspect'] = ''
# isDelays_df['sentiment'] = ''
# isCosts_df['sub_aspect'] = ''
# isCosts_df['sentiment'] = ''
# isAccess_df['sub_aspect'] = ''
# isAccess_df['sentiment'] = ''
# isErrors_df['sub_aspect'] = ''
# isErrors_df['sentiment'] = ''
# isTreatments_df['sub_aspect'] = ''
# isTreatments_df['sentiment'] = ''
# isStaffAndTrust_df['sub_aspect'] = ''
# isStaffAndTrust_df['sentiment'] = ''

isDelays_df['batch'] = [int(each / (isDelays_df.shape[0]/10)) + 1 for each in sample(range(0, isDelays_df.shape[0]), isDelays_df.shape[0])]
isCosts_df['batch'] = [int(each / (isCosts_df.shape[0]/10)) + 1 for each in sample(range(0, isCosts_df.shape[0]), isCosts_df.shape[0])]
isAccess_df['batch'] = [int(each / (isAccess_df.shape[0]/10)) + 1 for each in sample(range(0, isAccess_df.shape[0]), isAccess_df.shape[0])]
isErrors_df['batch'] = [int(each / (isErrors_df.shape[0]/10)) + 1 for each in sample(range(0, isErrors_df.shape[0]), isErrors_df.shape[0])]
isTreatments_df['batch'] = [int(each / (isTreatments_df.shape[0]/10)) + 1 for each in sample(range(0, isTreatments_df.shape[0]), isTreatments_df.shape[0])]
isStaffAndTrust_df['batch'] = [int(each / (isStaffAndTrust_df.shape[0]/10)) + 1 for each in sample(range(0, isStaffAndTrust_df.shape[0]), isStaffAndTrust_df.shape[0])]

train_path = 'train'
os.mkdir(train_path)
isDelays_df.to_excel(train_path + '/isDelays.xlsx', index=False)
isCosts_df.to_excel(train_path + '/isCosts_df.xlsx', index=False)
isAccess_df.to_excel(train_path + '/isAccess_df.xlsx', index=False)
isErrors_df.to_excel(train_path + '/isErrors_df.xlsx', index=False)
isTreatments_df.to_excel(train_path + '/isTreatments_df.xlsx', index=False)
isStaffAndTrust_df.to_excel(train_path + '/isStaffAndTrust_df.xlsx', index=False)

batchNum = 1
batch_path  = 'batch' + str(batchNum)
os.mkdir(batch_path)
isDelays_df[isDelays_df.batch == batchNum].to_excel(batch_path +'/isDelays_batch' + str(batchNum) + '.xlsx', index=False)
isCosts_df[isCosts_df.batch == batchNum].to_excel(batch_path +'/isCosts_batch'+ str(batchNum) + '.xlsx', index=False)
isAccess_df[isAccess_df.batch == batchNum].to_excel(batch_path + '/isAccess_batch' + str(batchNum) + '.xlsx', index=False)
isErrors_df[isErrors_df.batch == batchNum].to_excel(batch_path + '/isErrors_batch' + str(batchNum) + '.xlsx', index=False)
isTreatments_df[isTreatments_df.batch == batchNum].to_excel(batch_path + '/isTreatments_batch' + str(batchNum) + '.xlsx', index=False)
isStaffAndTrust_df[isStaffAndTrust_df.batch == batchNum].to_excel(batch_path + '/isStaffAndTrust_batch' + str(batchNum) + '.xlsx', index=False)

In [85]:
# generate 1000 sample for batch1
batchNum = 1
isDelays_df_batch1_rest, isDelays_df_batch1_1000 = train_test_split(isDelays_df[isDelays_df.batch == batchNum], test_size=1000, random_state=1)
isCosts_df_batch1_rest, isCosts_df_batch1_1000 = train_test_split(isCosts_df[isCosts_df.batch == batchNum], test_size=1000, random_state=1)
isAccess_df_batch1_rest, isAccess_df_batch1_1000 = train_test_split(isAccess_df[isAccess_df.batch == batchNum], test_size=1000, random_state=1)
isErrors_df_batch1_rest, isErrors_df_batch1_1000 = train_test_split(isErrors_df[isErrors_df.batch == batchNum], test_size=1000, random_state=1)
isTreatments_df_batch1_rest, isTreatments_df_batch1_1000 = train_test_split(isTreatments_df[isTreatments_df.batch == batchNum], test_size=1000, random_state=1)
isStaffAndTrust_df_batch1_rest, isStaffAndTrust_df_batch1_1000 = train_test_split(isStaffAndTrust_df[isStaffAndTrust_df.batch == batchNum], test_size=1000, random_state=1)

batch_path1000  = batch_path + '/batch1_1000' 
os.mkdir(batch_path1000)
isDelays_df_batch1_1000.to_excel(batch_path1000 + '/isDelays_batch1_1000.xlsx', index=False)
isCosts_df_batch1_1000.to_excel(batch_path1000 + '/isCosts_batch1_1000.xlsx', index=False)
isAccess_df_batch1_1000.to_excel(batch_path1000 +'/isAccess_batch1_1000.xlsx', index=False)
isErrors_df_batch1_1000.to_excel(batch_path1000 + '/isErrors_batch1_1000.xlsx', index=False)
isTreatments_df_batch1_1000.to_excel(batch_path1000 + '/isTreatments_batch1_1000.xlsx', index=False)
isStaffAndTrust_df_batch1_1000.to_excel(batch_path1000 + '/isStaffAndTrust_batch1_1000.xlsx', index=False)

batch_pathRest = batch_path + '/batch1_rest'
os.mkdir(batch_pathRest)
isDelays_df_batch1_rest.to_excel(batch_pathRest + '/isDelays_batch1_rest.xlsx', index=False)
isCosts_df_batch1_rest.to_excel(batch_pathRest + '/isCosts_batch1_rest.xlsx', index=False)
isAccess_df_batch1_rest.to_excel(batch_pathRest + '/isAccess_batch1_rest.xlsx', index=False)
isErrors_df_batch1_rest.to_excel(batch_pathRest + '/isErrors_batch1_rest.xlsx', index=False)
isTreatments_df_batch1_rest.to_excel(batch_pathRest + '/isTreatments_batch1_rest.xlsx', index=False)
isStaffAndTrust_df_batch1_rest.to_excel(batch_pathRest + '/isStaffAndTrust_batch1_rest.xlsx', index=False)